In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("202210 - EDW - CCU_assessment.csv")
df.shape

(3524, 7)

In [42]:
df

,MRN_hidden,ENC_NO_hidden,ADT_EVNT_NM,FROM_DEPT,ADT_DTTM_hidden,TO_DEPT,Unnamed: 6
0,203470,29204845127,TRANSFER OUT,7-OR,2022-05-19 11:54:00,4N-CICU,NaN
1,203470,29204845127,TRANSFER OUT,4N-CICU,2022-05-22 13:25:00,6-SE,NaN
2,219133,29202551651,TRANSFER OUT,ASAP EMERGENCY DEPT,2022-03-18 22:12:00,4N-CICU,NaN
3,219133,29202551651,TRANSFER OUT,4N-CICU,2022-03-19 22:04:00,6-SE,NaN
4,255606,29200807911,TRANSFER OUT,ASAP EMERGENCY DEPT,2022-01-28 14:40:00,4N-CICU,NaN
...,...,...,...,...,...,...,...
3519,203227827,29213895096,TRANSFER OUT,CARDIAC CATH LAB MAIN,2022-08-29 22:40:00,4N-CICU,NaN
3520,203227827,29213895096,TRANSFER OUT,4N-CICU,2022-08-30 18:23:00,3N-UNIV,NaN
3521,203228812,29213977480,TRANSFER OUT,CARDIAC CATH LAB MAIN,2022-08-30 18:17:00,4N-CICU,NaN
3522,990164086,29207074749,TRANSFER OUT,6-SE,2022-07-16 09:53:00,4N-CICU,NaN


In [4]:
df.groupby(['MRN_hidden', 'ENC_NO_hidden'])


In [5]:
# CICU or CCU
# admission to hospital is not necessarily ICU

# for first field, want ADMISSION and CICU and CCU to be in the
# if FROME_DEPT does not contain "CICU" or "CCU" and TO_DEPT contains "CICU" or "CCU" --> admission 

# Convert 'ADT_DTTM_hidden' to datetime
df['ADT_DTTM_hidden'] = pd.to_datetime(df['ADT_DTTM_hidden'])

# Filter rows where the event is an ICU admission
#icu_admissions = df[df['ADT_EVNT_NM'] == 'ADMISSION']

# create a df that contains only ICU visits
admission_condition = ((df['FROM_DEPT'].str.contains('CICU|CCU', na=False) == False) & 
                       (df['TO_DEPT'].str.contains('CICU|CCU', na=False) == True))

#| ((df['FROM_DEPT'].str.contains('CICU|CCU', na=False) == True)
                                                                                     #  & df['TO_DEPT'].str.contains('CICU|CCU', na=False) == True)
admission_df = df[admission_condition]
admission_df

# admission_df-->df that contains only ICU admissions

# want the ICU admission date for each hospital visit
first_icu_admission_dates = admission_df.groupby(['MRN_hidden', 'ENC_NO_hidden'])['ADT_DTTM_hidden'].min().reset_index()
first_icu_admission_dates = first_icu_admission_dates.rename(columns={'ADT_DTTM_hidden': 'FIRST_ICU_ADMISSION'})
first_icu_admission_dates
#first_icu_admission_dates[first_icu_admission_dates['MRN_hidden'] == 51455895] # verified results


,MRN_hidden,ENC_NO_hidden,FIRST_ICU_ADMISSION
0,203470,29204845127,2022-05-19 11:54:00
1,219133,29202551651,2022-03-18 22:12:00
2,255606,29200807911,2022-01-28 14:40:00
3,270484,29207707039,2022-07-30 20:13:00
4,282052,29191878193,2021-05-26 08:45:00
...,...,...,...
1191,203226201,29213550718,2022-08-25 21:52:00
1192,203226240,29213552992,2022-08-26 05:59:00
1193,203227827,29213895096,2022-08-29 08:01:00
1194,203228812,29213977480,2022-08-30 18:17:00


In [6]:
first_icu_admission_dates['MRN_hidden'].unique().shape

(1141,)

In [51]:
first_icu_admission_dates.shape # patient can be admitted to be 
# dataframe currently contains duplicate patients (which is correct)
# patient can have multiple hospital visits--2 patients can have the same encounter number

(1196, 3)

In [7]:
admission_df['ENC_NO_hidden'].unique().shape

(1196,)

In [8]:
admission_df.shape

(1535, 7)

In [9]:
# subset df of patients with first ICU discharge
discharge_condition = (df['FROM_DEPT'].str.contains('CICU|CCU', na=False) == True) & (df['TO_DEPT'].str.contains('CICU|CCU', na=False) == False)
discharge_df = df[discharge_condition]
discharge_df


,MRN_hidden,ENC_NO_hidden,ADT_EVNT_NM,FROM_DEPT,ADT_DTTM_hidden,TO_DEPT,Unnamed: 6
1,203470,29204845127,TRANSFER OUT,4N-CICU,2022-05-22 13:25:00,6-SE,NaN
3,219133,29202551651,TRANSFER OUT,4N-CICU,2022-03-19 22:04:00,6-SE,NaN
5,255606,29200807911,TRANSFER OUT,4N-CICU,2022-01-31 12:18:00,3N-UNIV,NaN
7,270484,29207707039,DISCHARGE,4N-CICU,2022-07-31 09:25:00,?,NaN
9,282052,29191878193,TRANSFER OUT,4N-CICU,2021-05-26 23:10:00,5-NE,NaN
...,...,...,...,...,...,...,...
3515,203227786,29213892603,ADMISSION,4N-CICU,2022-08-31 00:47:00,?,NaN
3516,203227786,29213892603,TRANSFER OUT,4N-CICU,2022-08-31 16:22:00,6-SW,NaN
3518,203227827,29213895096,TRANSFER OUT,4N-CICU,2022-08-29 20:25:00,CARDIAC CATH LAB MAIN,NaN
3520,203227827,29213895096,TRANSFER OUT,4N-CICU,2022-08-30 18:23:00,3N-UNIV,NaN


In [ ]:
# to address note 1, remove entries if LOS <= 1 and FROM_DEPT and TO_DEPT contains ICU
# create a separate df where FROM_DEPT and TO_DEPT both contain ICU --> both_ICU
# compute LOS, append this col to both_ICU
# if LOS <= 1 for both_ICU df, remove such entries
# update both ICU and add as separate entries

In [ ]:
# to address note 2, addmitted ICU and still there 
# any that were not successfully joined applies to this, create separate entries ??

In [10]:
#3485
first_icu_discharge_dates = discharge_df.groupby(['MRN_hidden', 'ENC_NO_hidden'])['ADT_DTTM_hidden'].min().reset_index()
first_icu_discharge_dates = first_icu_discharge_dates.rename(columns={'ADT_DTTM_hidden': 'FIRST_ICU_DISCHARGE'})
first_icu_discharge_dates

# not getting the same rows for the first admission and first discharge
#first_icu_discharge_dates.tail



,MRN_hidden,ENC_NO_hidden,FIRST_ICU_DISCHARGE
0,203470,29204845127,2022-05-22 13:25:00
1,219133,29202551651,2022-03-19 22:04:00
2,255606,29200807911,2022-01-31 12:18:00
3,270484,29207707039,2022-07-31 09:25:00
4,282052,29191878193,2021-05-26 23:10:00
...,...,...,...
1322,203226240,29213552992,2022-08-27 08:32:00
1323,203226941,29213814414,2022-08-28 18:40:00
1324,203227786,29213892603,2022-08-31 00:47:00
1325,203227827,29213895096,2022-08-29 20:25:00


In [11]:
LOS_df = pd.merge(first_icu_admission_dates, first_icu_discharge_dates, on=['MRN_hidden', 'ENC_NO_hidden'], how='inner')

In [12]:
LOS_df

,MRN_hidden,ENC_NO_hidden,FIRST_ICU_ADMISSION,FIRST_ICU_DISCHARGE
0,203470,29204845127,2022-05-19 11:54:00,2022-05-22 13:25:00
1,219133,29202551651,2022-03-18 22:12:00,2022-03-19 22:04:00
2,255606,29200807911,2022-01-28 14:40:00,2022-01-31 12:18:00
3,270484,29207707039,2022-07-30 20:13:00,2022-07-31 09:25:00
4,282052,29191878193,2021-05-26 08:45:00,2021-05-26 23:10:00
...,...,...,...,...
1187,203226114,29213544939,2022-08-28 20:38:00,2022-08-29 10:23:00
1188,203226201,29213550718,2022-08-25 21:52:00,2022-08-27 12:42:00
1189,203226240,29213552992,2022-08-26 05:59:00,2022-08-27 08:32:00
1190,203227827,29213895096,2022-08-29 08:01:00,2022-08-29 20:25:00


In [24]:
LOS_df['FIRST_ICU_LOS'] = LOS_df['FIRST_ICU_DISCHARGE'] - LOS_df['FIRST_ICU_ADMISSION'] 
LOS_df = LOS_df.rename(columns={'MRN_hidden': 'MRN', 'ENC_NO_hidden': 'ENC_NO'})
LOS_df # final answer

,MRN,ENC_NO,FIRST_ICU_ADMISSION,FIRST_ICU_DISCHARGE,FIRST_ICU_LOS
0,203470,29204845127,2022-05-19 11:54:00,2022-05-22 13:25:00,3 days 01:31:00
1,219133,29202551651,2022-03-18 22:12:00,2022-03-19 22:04:00,0 days 23:52:00
2,255606,29200807911,2022-01-28 14:40:00,2022-01-31 12:18:00,2 days 21:38:00
3,270484,29207707039,2022-07-30 20:13:00,2022-07-31 09:25:00,0 days 13:12:00
4,282052,29191878193,2021-05-26 08:45:00,2021-05-26 23:10:00,0 days 14:25:00
...,...,...,...,...,...
1187,203226114,29213544939,2022-08-28 20:38:00,2022-08-29 10:23:00,0 days 13:45:00
1188,203226201,29213550718,2022-08-25 21:52:00,2022-08-27 12:42:00,1 days 14:50:00
1189,203226240,29213552992,2022-08-26 05:59:00,2022-08-27 08:32:00,1 days 02:33:00
1190,203227827,29213895096,2022-08-29 08:01:00,2022-08-29 20:25:00,0 days 12:24:00


In the cells above, I split the dataframe into two--one containing earliest ICU admission and the second containing earliest ICU discharge. I then join the two dataframes based on shared values in the columns MRN_hidden and ENC_NO_hidden. I do an inner join to specify only values in MRN_hidden AND ENC_NO_hidden shared by the earliest discharge and earliest ICU discharge will be included in the output dataframe. That way I can do can subtract the FIRST_ICU_DISCHARGE and FIRST_ICU_ADMISSION columns easily to get length of stay.
Observed that dataframe currently contains duplicate patients (which is correct). And patient can have multiple hospital visits--2 patients can have the same encounter number